<h2>Load the data</h2>

In [1]:
import os
import re
import sys
import glob
import numpy as np
import pandas as pd
from nltk import tokenize

In [23]:
txt_files = []
for name in glob.glob('ElecDeb60To16/*.txt'):
    print(name)
    name = name.replace('\\','/')
    txt_files.append(name)

ann_files = []
for name in glob.glob('ElecDeb60To16/*.ann'):
    name = name.replace('\\','/')
    ann_files.append(name)

ElecDeb60To16\01_1960.txt
ElecDeb60To16\02_1960.txt
ElecDeb60To16\03_1960.txt
ElecDeb60To16\04_1960.txt
ElecDeb60To16\05_1976.txt
ElecDeb60To16\06_1976.txt
ElecDeb60To16\07_1976.txt
ElecDeb60To16\08_1980.txt
ElecDeb60To16\09_1980.txt
ElecDeb60To16\10_1984.txt
ElecDeb60To16\11_1984.txt
ElecDeb60To16\12_1984.txt
ElecDeb60To16\13_1988.txt
ElecDeb60To16\14_1988.txt
ElecDeb60To16\15_1988.txt
ElecDeb60To16\16_1992.txt
ElecDeb60To16\17_1992.txt
ElecDeb60To16\18_1992.txt
ElecDeb60To16\19_1992.txt
ElecDeb60To16\20_1992.txt
ElecDeb60To16\21_1992.txt
ElecDeb60To16\22_1996.txt
ElecDeb60To16\23_1996.txt
ElecDeb60To16\24_1996.txt
ElecDeb60To16\25_2000.txt
ElecDeb60To16\26_2000.txt
ElecDeb60To16\27_2000.txt
ElecDeb60To16\28_2000.txt
ElecDeb60To16\29_2004.txt
ElecDeb60To16\30_2004.txt
ElecDeb60To16\31_2004.txt
ElecDeb60To16\32_2004.txt
ElecDeb60To16\33_2008.txt
ElecDeb60To16\34_2008.txt
ElecDeb60To16\35_2008.txt
ElecDeb60To16\36_2008.txt
ElecDeb60To16\37_2012.txt
ElecDeb60To16\38_2012.txt
ElecDeb60To1

In [24]:
sent_all = []
ind_all = []
for file in txt_files:
    sent_list = []
    ind_list = []
    prev_ind = 0
    
    txt_file = open(file)
    text = txt_file.read()
    txt_file.close()
    
    
    with open(file) as infile:
        for line in infile:
            line = re.sub(r'(\([A-Z]+\)_[A-Z]+(\_[A-Z]+)?: )','',line)
            line = re.sub(r'([A-Z]+\_[A-Z]+\_[A-Z]+: )','',line)
            line = re.sub(r'([A-Z]+\_[A-Z]+: )','',line)
            line = re.sub(r'(\_[A-Z]+: )','',line)
            line = re.sub(r'([A-Z]+: )','',line)
            line = re.sub(r'\[[a-zA-Z0-9;,\s\"\'\.\S]*\]*','',line)
                  
            for sent in tokenize.sent_tokenize(line):
                if not sent_list:
                    ind = text.find(sent,0,len(text)) 
                else:
                    ind = text.find(sent,prev_ind,len(text))
                prev_ind = ind
                sent_list.append([sent,ind])
                ind_list.append(ind)
                
    ind_all.append(np.array(ind_list))
    sent_all.append(sent_list)

#Up to this point we get a sent_all containing every debate (42 total) tokenized into sentences.
#sent_all = [sent_list, sent_list, sent_list,...]
#sent_list = [[sent1, index],[sent2, index],[sent3,index],...]
#ind_all = [ind_list, ind_list, ind_list,...]
#ind_list = [ind, ind, ind,...]

In [25]:
print(ind_all)

[array([    7,    21,   247,   284,   398,   599,   709,   879,   956,
         978,  1098,  1375,  1538,  1717,  1870,  1929,  2037,  2183,
        2308,  2340,  2438,  2472,  2693,  2837,  2986,  3061,  3201,
        3351,  3626,  3843,  3948,  4067,  4128,  4246,  4354,  4671,
        4790,  4868,  5051,  5126,  5230,  5331,  5357,  5414,  5494,
        5510,  5565,  5619,  5672,  5772,  5894,  6143,  6394,  6477,
        6566,  6616,  6648,  6783,  6827,  6894,  6970,  7094,  7156,
        7191,  7427,  7471,  7586,  7647,  7749,  7768,  7835,  7890,
        7963,  7991,  8059,  8236,  8413,  8488,  8560,  8698,  8727,
        8928,  9078,  9114,  9152,  9193,  9314,  9337,  9546,  9583,
        9717,  9878,  9923, 10062, 10211, 10231, 10389, 10421, 10558,
       10581, 10675, 10705, 10758, 11093, 11182, 11255, 11281, 11412,
       11443, 11577, 11608, 11743, 11776, 11838, 11879, 11907, 12016,
       12087, 12152, 12220, 12324, 12420, 12592, 12626, 12836, 12997,
       13091, 13128

In [26]:
ann_all = []

for file in ann_files:
    ann_list = []
    
    with open(file) as infile:
        for line in infile:
            entry = line.split("\t")
            ann_list.append(entry)
    
    ann_all.append(ann_list)

In [27]:
#following part creates labels for task 1 (whether a sentence contains an argument component)

labels_arg_all = []

for i in range(len(ind_all)):
    labels_arg_arr = np.empty([len(ind_all[i]),2])
    sent_start_ind_list = []

    #column 0 of labels_arg are the starting indices of the senteces 
    for j in range(len(ind_all[i])):
        labels_arg_arr[j,0] = ind_all[i][j]
        
    #loop through each annotation entry, get the corresponding sentence starting index 
    for j in range(len(ann_all[i])):
        phrase_start = re.search(r'[0-9]+\s',ann_all[i][j][1]).group(0)
        sent_start_ind_list.append(ind_all[i][ind_all[i] <= int(phrase_start)].max())
        
        phrase_start = re.search(r';[0-9]+\s',ann_all[i][j][1])
        if phrase_start != None:
            phrase_start = phrase_start.group(0)[1:]
            ind2 = ind_all[i][ind_all[i] <= int(phrase_start)].max()
            if ind2 not in sent_start_ind_list:
                sent_start_ind_list.append(ind2)
    
    #loop through starting indices for sentences, check if each sentence contains an annotation (labeled as 1)
    for j in range(len(ind_all[i])):   
        if ind_all[i][j] in sent_start_ind_list:
            labels_arg_arr[j,1] = 1
        else:
            labels_arg_arr[j,1] = 0
    
    labels_arg_all.append(labels_arg_arr.astype(int))

        
#ann_all = [ann_list, ann_list, ann_list, ...]
#ann_list = [['T1', 'Claim/Premise xxx xxx', '...'], ['T2', 'Claim/Premise xxx xxx', '...'], 
    #['T3', 'Claim/Premise xxx xxx', '...'], ...]
#labels_arg_all = [labels_arg_arr, labels_arg_arr, labesl_arg_arr, ...]
#labels_arg_arr = np array of size (len(ind_list),2), no. of rows = no. of sentences in a txt file, 
    #col 0 = starting index of sentece, col 1 = label (1 for contain claim/premise, 0 for otherwise)

In [28]:
labels_cp_all = []

for i in range(len(labels_arg_all)):
    labels_cp_list = []
    
    for j in range(labels_arg_all[i].shape[0]):
        if labels_arg_all[i][j,1] != 0:
            labels_cp_list.append(labels_arg_all[i][j,0])
    
    labels_cp_arr = np.zeros((len(labels_cp_list),3))
    labels_cp_out = np.zeros((len(labels_cp_list),2))
    
    for k in range(len(labels_cp_list)):
        labels_cp_arr[k,0] = labels_cp_list[k]

        
    for j in range(len(ann_all[i])):
        #check for claim with only 1 continuous part
        phrase_start = re.search(r'c|Claim\s[0-9]+\s[0-9]+',ann_all[i][j][1])
        if phrase_start != None:
            phrase_start = phrase_start.group(0)
            labels_cp_arr[np.where(labels_cp_arr[:,0] == ind_all[i][ind_all[i] <= int(phrase_start.split()[1])].max())[0],1] +=\
            int(phrase_start.split()[2]) - int(phrase_start.split()[1])
        
        #check for claim formed from 2 separate parts
        phrase_start = re.search(r'c|Claim[0-9\s]+;[0-9]+\s[0-9]+',ann_all[i][j][1])
        if phrase_start != None:
            phrase_start = phrase_start.group(0)
            labels_cp_arr[np.where(labels_cp_arr[:,0] == ind_all[i][ind_all[i] <= int(phrase_start.split(';')[1].split()[0])].max())[0],1] +=\
            int(phrase_start.split(';')[1].split()[1]) - int(phrase_start.split(';')[1].split()[0])
        
        #check for premise with only 1 continuous part
        phrase_start = re.search(r'p|Premise\s[0-9]+\s[0-9]+',ann_all[i][j][1])
        if phrase_start != None:
            phrase_start = phrase_start.group(0)
            labels_cp_arr[np.where(labels_cp_arr[:,0] == ind_all[i][ind_all[i] <= int(phrase_start.split()[1])].max())[0],2] +=\
            int(phrase_start.split()[2]) - int(phrase_start.split()[1])
        
        #check for premise formed from 2 separate parts
        phrase_start = re.search(r'p|Premise[0-9\s]+;[0-9]+\s[0-9]+',ann_all[i][j][1])
        if phrase_start != None:
            phrase_start = phrase_start.group(0)
            labels_cp_arr[np.where(labels_cp_arr[:,0] == ind_all[i][ind_all[i] <= int(phrase_start.split(';')[1].split()[0])].max())[0],2] +=\
            int(phrase_start.split(';')[1].split()[1]) - int(phrase_start.split(';')[1].split()[0])
    
    #compare length of claim and premise components in a sentence, and label the sentence according to the longer component
    labels_cp_out[:,0] = labels_cp_arr[:,0]
    for m in range(labels_cp_arr.shape[0]):
        if labels_cp_arr[m,1] >= labels_cp_arr[m,2]:
            labels_cp_out[m,1] = 1
            
    labels_cp_all.append(labels_cp_out)
    
#labels_cp_all = [labels_cp_out, labels_cp_out, labels_cp_out, ...]
#labels_cp_out = np array of size ((len(labels_cp_list),3)), 
    #no. of rows = no. sentences that contains at least 1 claim or 1 premise,
    #col 0 = starting index of such a sentence, col 1 = label (1 for claim, 0 for premise)

In [29]:
np.set_printoptions(threshold=sys.maxsize)
print(labels_arg_all[0:50])

[array([[    7,     0],
       [   21,     0],
       [  247,     0],
       [  284,     0],
       [  398,     0],
       [  599,     0],
       [  709,     0],
       [  879,     0],
       [  956,     0],
       [  978,     0],
       [ 1098,     1],
       [ 1375,     1],
       [ 1538,     1],
       [ 1717,     1],
       [ 1870,     1],
       [ 1929,     1],
       [ 2037,     1],
       [ 2183,     0],
       [ 2308,     0],
       [ 2340,     1],
       [ 2438,     0],
       [ 2472,     0],
       [ 2693,     1],
       [ 2837,     1],
       [ 2986,     1],
       [ 3061,     1],
       [ 3201,     1],
       [ 3351,     1],
       [ 3626,     1],
       [ 3843,     1],
       [ 3948,     1],
       [ 4067,     1],
       [ 4128,     1],
       [ 4246,     1],
       [ 4354,     1],
       [ 4671,     0],
       [ 4790,     1],
       [ 4868,     1],
       [ 5051,     1],
       [ 5126,     1],
       [ 5230,     1],
       [ 5331,     1],
       [ 5357,     1],
       [ 5

In [30]:
ints = []
for lst in labels_cp_all:
    for char in lst:
        ints.append([int(char[0]), int(char[1])])

In [31]:
sentences = []
for lst in sent_all:
    for lsts in lst:
        sentences.append(lsts[0])

In [32]:
import csv
from itertools import zip_longest
indx = []
annotation = []
changed = []
# looping through labels and 
for i in labels_arg_all:
    for j in i:
        # if 0 - not annotated
        if j[1] == 0: 
            annotation.append([j[0], 'None'])
        # if 1 - annotated
        elif j[1] == 1:
            annotation.append([j[0], 'annotated'])
        indx.append(j[0])

In [33]:
claim_or_premise = {}
for char in ints:
    for num in indx:
        if int(char[0]) ==  num:
            claim_or_premise[char[0]] = char[1]
        else:
            continue

In [34]:
cp = {}
for k, v in claim_or_premise.items():
    if v == 0:
        cp[k] = 'Premise'
    elif v == 1:
        cp[k] = 'Claim'

In [35]:
dict_annotation = {}
dict_cp = {}
final = []
for i in annotation:
    dict_annotation[i[0]] = i[1]

In [36]:
print(dict_annotation)

{7: 'None', 21: 'None', 247: 'None', 284: 'None', 398: 'None', 599: 'None', 709: 'None', 879: 'None', 956: 'annotated', 978: 'None', 1098: 'annotated', 1375: 'annotated', 1538: 'annotated', 1717: 'annotated', 1870: 'annotated', 1929: 'annotated', 2037: 'None', 2183: 'None', 2308: 'None', 2340: 'annotated', 2438: 'None', 2472: 'None', 2693: 'annotated', 2837: 'annotated', 2986: 'annotated', 3061: 'annotated', 3201: 'annotated', 3351: 'None', 3626: 'annotated', 3843: 'annotated', 3948: 'annotated', 4067: 'annotated', 4128: 'annotated', 4246: 'annotated', 4354: 'annotated', 4671: 'None', 4790: 'annotated', 4868: 'annotated', 5051: 'annotated', 5126: 'annotated', 5230: 'annotated', 5331: 'annotated', 5357: 'annotated', 5414: 'None', 5494: 'annotated', 5510: 'annotated', 5565: 'annotated', 5619: 'annotated', 5672: 'None', 5772: 'None', 5894: 'None', 6143: 'None', 6394: 'None', 6477: 'annotated', 6566: 'None', 6616: 'annotated', 6648: 'annotated', 6783: 'None', 6827: 'None', 6894: 'annotated

In [39]:
final = {}
for k, v in dict_annotation.items():
    if k not in cp.keys():
        final[k] = v
    else:
        final[k] = v
for k, v in cp.items():
    if k not in final.keys():
        final[k] = v
    else:
        final[k] = [final[k], v] 
print(final)

{7: ['None', 'Claim'], 21: 'None', 247: 'None', 284: 'None', 398: 'None', 599: 'None', 709: 'None', 879: 'None', 956: ['annotated', 'Claim'], 978: 'None', 1098: ['annotated', 'Premise'], 1375: ['annotated', 'Claim'], 1538: ['annotated', 'Claim'], 1717: ['annotated', 'Premise'], 1870: ['annotated', 'Premise'], 1929: ['annotated', 'Premise'], 2037: ['None', 'Claim'], 2183: 'None', 2308: 'None', 2340: ['annotated', 'Claim'], 2438: 'None', 2472: 'None', 2693: ['annotated', 'Claim'], 2837: ['annotated', 'Claim'], 2986: ['annotated', 'Claim'], 3061: ['annotated', 'Claim'], 3201: ['annotated', 'Premise'], 3351: ['None', 'Premise'], 3626: ['annotated', 'Premise'], 3843: ['annotated', 'Claim'], 3948: ['annotated', 'Claim'], 4067: ['annotated', 'Claim'], 4128: ['annotated', 'Claim'], 4246: ['annotated', 'Claim'], 4354: ['annotated', 'Claim'], 4671: 'None', 4790: ['annotated', 'Claim'], 4868: ['annotated', 'Premise'], 5051: ['annotated', 'Premise'], 5126: ['annotated', 'Premise'], 5230: ['annotat

In [38]:
for k, v in final.items():
    #print(v)
    if v != 'None':
        for i in v:
            if i == 'Premise' or i == 'Claim':
                final[k] = i
    else:
        final[k] = v
print(final)

{7: 'Claim', 21: 'None', 247: 'None', 284: 'None', 398: 'None', 599: 'None', 709: 'None', 879: 'None', 956: 'Claim', 978: 'None', 1098: 'Premise', 1375: 'Claim', 1538: 'Claim', 1717: 'Premise', 1870: 'Premise', 1929: 'Premise', 2037: 'Claim', 2183: 'None', 2308: 'None', 2340: 'Claim', 2438: 'None', 2472: 'None', 2693: 'Claim', 2837: 'Claim', 2986: 'Claim', 3061: 'Claim', 3201: 'Premise', 3351: 'Premise', 3626: 'Premise', 3843: 'Claim', 3948: 'Claim', 4067: 'Claim', 4128: 'Claim', 4246: 'Claim', 4354: 'Claim', 4671: 'None', 4790: 'Claim', 4868: 'Premise', 5051: 'Premise', 5126: 'Premise', 5230: 'Premise', 5331: 'Claim', 5357: 'Claim', 5414: 'None', 5494: 'Premise', 5510: 'Claim', 5565: 'Claim', 5619: 'Claim', 5672: 'None', 5772: 'None', 5894: 'None', 6143: 'None', 6394: 'Claim', 6477: 'Claim', 6566: 'None', 6616: 'Claim', 6648: 'Premise', 6783: 'None', 6827: 'None', 6894: 'Claim', 6970: 'None', 7094: 'Premise', 7156: 'Claim', 7191: 'Claim', 7427: 'Claim', 7471: 'Premise', 7586: 'Claim',

In [20]:
export_data = zip_longest(indx, sentences, final.values(), fillvalue = '')
with open('try.csv', 'w', encoding="UTF-8", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow(("Index", "Sentences", "Annotation"))
      wr.writerows(export_data)
myfile.close()

In [21]:
import pandas
df = pandas.read_csv('try.csv')
print(df)

     Index                                          Sentences Annotation
0        7                                      Good evening.       None
1       21  The television and radio stations of the Unite...       None
2      247               The candidates need no introduction.       None
3      284  The Republican candidate, Vice President Richa...       None
4      398  According to rules set by the candidates thems...       None
..     ...                                                ...        ...
527  60222  Thank you very much for permitting us to prese...       None
528  60335  I've been asked by the candidates to thank the...       None
529  60489  Other debates in this series will be announced...       None
530  60577                           This is Howard K. Smith.       None
531  60602                           Good night from Chicago.        NaN

[532 rows x 3 columns]
